## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Pushover at: https://pushover.net/

Pushover is a simple and reliable push notification service.

Create a free account if you don't already have one.

Once logged in:
1. Create a new application at https://pushover.net/apps/build
2. Note your application's "API Token/Key"
3. Note your "User Key" from the main page

Add these keys to your .env file:

`PUSHOVER_TOKEN=xxxx`
`PUSHOVER_USER=xxxx`


In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import requests
import os
import asyncio



In [ ]:
load_dotenv(override=True)

In [ ]:
# Let's just check notifications are working for you

def send_test_notification():
    """Send a test notification via Pushover"""
    url = "https://api.pushover.net/1/messages.json"
    data = {
        "token": os.environ.get('PUSHOVER_TOKEN'),
        "user": os.environ.get('PUSHOVER_USER'),
        "message": "This is an important test notification",
        "title": "Test notification"
    }
    response = requests.post(url, data=data)
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")

send_test_notification()

### Did you receive the test notification?

If you get status 200 and {"status": 1, "request": "..."}, then everything works perfectly!

#### Possible errors

If you have errors, check:
1. That your PUSHOVER_TOKEN and PUSHOVER_USER keys are correctly defined in your .env
2. That you have created an application on Pushover and copied the right token
3. That your User Key is correct (visible on Pushover's main page)

#### Dependencies installation

If you don't have `requests` yet, install it:
```
!uv pip install requests
```

Pushover is much simpler and more reliable than email services for notifications!

## Step 1: Agent workflow

In [ ]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [ ]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [ ]:

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

In [ ]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


In [ ]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [ ]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [ ]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [ ]:
sales_agent1

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [ ]:
@function_tool
def send_notification(body: str):
    """ Send a notification with the given body to all sales prospects """
    url = "https://api.pushover.net/1/messages.json"
    data = {
        "token": os.environ.get('PUSHOVER_TOKEN'),
        "user": os.environ.get('PUSHOVER_USER'),
        "message": body,
        "title": "Sales Email"
    }
    response = requests.post(url, data=data)
    if response.status_code == 200:
        return {"status": "success", "message": "Notification sent successfully"}
    else:
        return {"status": "error", "message": f"Error {response.status_code}: {response.text}"}

### This has automatically been converted into a tool, with the boilerplate json created

In [ ]:
# Let's look at it
send_notification

### And you can also convert an Agent into a tool

In [ ]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [ ]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_notification]

tools

## And now it's time for our Sales Manager - our planning agent

In [ ]:
# Improved instructions thanks to Jessica Kuijer

instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_notification tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE notification using the send_notification tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get a notification??</h2>
            <span style="color:#ff7800;">If you don't receive a notification after running the previous cell, here are some things to check: <br/>
            1. Check that your PUSHOVER_TOKEN and PUSHOVER_USER keys are correctly defined<br/>
            2. Check your Pushover mobile app<br/>
            3. Look at the trace in OpenAI to see if there are any errors<br/>
            Pushover is much more reliable than emails - notifications usually arrive within seconds!
            </span>
        </td>
    </tr>
</table>

### Advantages of Pushover over emails:

- **Instant**: notifications arrive within seconds
- **Reliable**: no spam or SSL certificate issues
- **Simple**: easier setup with just two API keys
- **Multi-platform**: works on mobile, desktop and web
- **No complex sending limits** like with email services

Pushover is perfect for development notifications and alerts!

## Remember to check the trace

https://platform.openai.com/traces

And then check your notification!!


### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [ ]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [ ]:
@function_tool
def send_simple_notification(message: str) -> Dict[str, str]:
    """ Send a simple notification with the given message to all sales prospects """
    url = "https://api.pushover.net/1/messages.json"
    
    data = {
        "token": os.environ.get('PUSHOVER_TOKEN'),
        "user": os.environ.get('PUSHOVER_USER'),
        "message": message,
        "title": "Sales Email"
    }
    
    response = requests.post(url, data=data)
    if response.status_code == 200:
        return {"status": "success", "message": "Notification sent successfully"}
    else:
        return {"status": "error", "message": f"Error {response.status_code}: {response.text}"}

In [ ]:
tools = [send_simple_notification]

In [ ]:
tools

In [ ]:
instructions = "You are a notification sender. You receive the body of an email to be sent. \
You use the send_simple_notification tool to send the email content as a clean, readable notification. \
Do not add any formatting or subjects - just send the email content directly."


notification_agent = Agent(
    name="Notification Sender",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Send the email content as a simple notification")


### Now we have 3 tools and 1 handoff

In [ ]:
# Only the sales agent tools for the manager - NO send_notification tool
manager_tools = [tool1, tool2, tool3]
handoffs = [notification_agent]
print("Manager tools:", manager_tools)
print("Handoffs:", handoffs)

In [ ]:
# Improved instructions thanks to Jessica Kuijer

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: DO NOT use any send_notification tools yourself. Instead, hand off to the 'Notification Sender' agent with ONLY the winning email draft text. The Notification Sender will handle the actual sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must NEVER use send_notification tools - only hand off to the Notification Sender agent.
- You must hand off exactly ONE email to the Notification Sender — never more than one.
- When handing off, pass only the email text content, not multiple emails.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=manager_tools,  # Only sales agent tools, NO send_notification
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

### Remember to check the trace

https://platform.openai.com/traces

And then check your Pushover notifications!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google has released their Agent Development Kit (ADK). It's not yet got the traction of the other frameworks on this course, but it's getting some attention. It's interesting to note that it looks quite similar to OpenAI Agents SDK. To give you a preview, here's a peak at sample code from ADK:

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

